# ASP solving with clingo in Python

This is a small example to show how you can do Answer Set Programming with [clingo](https://potassco.org/clingo/) (part of the Potsdam Answer Set Solving Collection; or *Potassco*, for short) in Python.

First, install clingo, e.g., using:
`conda install -c potassco clingo`

To use clingo in Python, import the clingo package.

In [1]:
import clingo

In [5]:
!python --version

Python 3.11.7


You can then write an answer set program as a string.

For an explanation of the syntax of answer set programs (that clingo uses) and for examples, see Potassco's [Getting Started page](https://potassco.org/doc/start/) and their [guide](https://github.com/potassco/guide/releases/tag/v2.2.0).

Let's take the following simple example.

In [6]:
asp_program = """
    #const k=3.
    number(1..k).
    left(X) :- not right(X), number(X).
    right(X) :- not left(X), number(X).
    :- right(2).
"""

We then create a clingo Control object, load the answer set program, and do the grounding (compiling away variables).

In [2]:
control = clingo.Control()
control.add("base", [], asp_program)
control.ground([("base", [])])

NameError: name 'asp_program' is not defined

Before we ask clingo to find models (answer sets) for our program, we define a function `on_model()` that will be called for each model.

For this example, we let this function extract and print some information about literals in the answer set that use the predicates `left/1` and `right/1`.

In [8]:
def on_model(model):
    print("ANSWER SET:")
    for atom in model.symbols(atoms=True):
        if atom.name == "right":
            print("Right: {}".format(atom.arguments[0].number))
        elif atom.name == "left":
            print("Left: {}".format(atom.arguments[0].number))

We then ask clingo to find a two answer sets for our program, and to call `on_model` on these when they are found.

In [9]:
control.configuration.solve.models = 2  # use 0 if you want to find all models
answer = control.solve(on_model=on_model)

ANSWER SET:
Right: 1
Right: 3
Left: 2
ANSWER SET:
Right: 1
Left: 2
Left: 3


We can check whether `solve()` found a model or not:

In [10]:
if answer.satisfiable:
    print("Found at least one answer set!")
else:
    print("Did not find an answer set!")

Found at least one answer set!


## Yielding answer sets

Alternatively, instead of using the function `on_model()`, we can iterate over all answer sets. This works as follows.

ex1_a
```
 a.
 b :- a, not c, not d.
 c :- a, not b, not d.
 d :- a, not b, not c.
```
ex1_b
```
 a :- not b.
 b :- not a.
 c :- not d.
 d :- not c.
```
ex1_c
```
 a.
```

In [88]:
import matplotlib.pyplot as plt
import numpy as np


def plot_bounding_boxes(bounding_boxes):
    fig, ax = plt.subplots()
    num_boxes = len(bounding_boxes)
    colors = plt.cm.rainbow(np.linspace(0, 1, num_boxes))

    for i, box in enumerate(bounding_boxes):
        y_min, y_max, x_min, x_max = box
        width = x_max - x_min
        height = y_max - y_min
        color = colors[i]
        rect = plt.Rectangle(
            (x_min, y_min),
            width,
            height,
            linewidth=1,
            color=color,
        )
        ax.add_patch(rect)

    # Set aspect ratio to 'equal' to maintain square ticks
    ax.set_aspect("equal")
    ax.autoscale_view()
    plt.yticks(np.arange(int(ax.get_ylim()[0]), int(ax.get_ylim()[1]) + 1, 1))
    plt.xlabel("X-axis")
    plt.ylabel("Y-axis")
    plt.title("Bounding Boxes")
    plt.grid(True)
    plt.show()

In [1]:
import clingo

with open("truck_delivery.lp", "r") as file:
    new_program = file.read()

In [1]:
import clingo

with open("truck_delivery.lp", "r") as file:
    new_program = file.read()


def on_model(model):
    sorted_model = [str(atom) for atom in model.symbols(shown=True)]
    sorted_model.sort()
    if sorted_model:
        print("Answer set: {{{}}}".format(", ".join(sorted_model)))


control = clingo.Control()
control.add("base", [], new_program)
control.ground([("base", [])])

control.configuration.solve.models = 0

with control.solve(yield_=True, on_model=on_model) as handle:
    count = 0
    for model in handle:
        count += 1
        bounding_boxes = []
        for item in model.symbols(shown=True):
            if item.name == "positions":
                bounding_boxes.append(
                    [
                        item.arguments[1].number,
                        item.arguments[2].number,
                        item.arguments[3].number,
                        item.arguments[4].number,
                    ]
                )
        if bounding_boxes:
            bounding_boxes.sort()
            plot_bounding_boxes(bounding_boxes)
    if handle.get().satisfiable:
        print(f"Found {count} answer set{'' if count == 1 else 's'}!")
    else:
        print("Did not find an answer set!")
    if count < 25:
        pass

Answer set: {do(1,1,move(2)), do(1,2,move(6)), do(10,1,move(3)), do(10,2,move(2)), do(11,1,move(2)), do(11,2,move(1)), do(12,1,move(3)), do(12,2,load), do(13,1,move(2)), do(13,2,wait), do(14,1,move(3)), do(14,2,wait), do(15,1,move(4)), do(15,2,wait), do(16,1,move(5)), do(16,2,move(2)), do(17,1,move(4)), do(17,2,move(3)), do(18,1,wait), do(18,2,wait), do(19,1,move(5)), do(19,2,move(4)), do(2,1,wait), do(2,2,wait), do(20,1,wait), do(20,2,unload), do(21,1,wait), do(21,2,wait), do(22,1,wait), do(22,2,move(3)), do(23,1,move(4)), do(23,2,move(2)), do(24,1,move(3)), do(24,2,move(6)), do(25,1,move(4)), do(25,2,move(3)), do(26,1,move(5)), do(26,2,move(4)), do(27,1,move(4)), do(27,2,move(3)), do(28,1,wait), do(28,2,move(2)), do(29,1,move(3)), do(29,2,charge), do(3,1,wait), do(3,2,move(3)), do(30,1,move(6)), do(30,2,wait), do(31,1,wait), do(31,2,move(1)), do(32,1,move(2)), do(32,2,wait), do(33,1,move(3)), do(33,2,load), do(34,1,move(4)), do(34,2,move(2)), do(35,1,move(5)), do(35,2,move(3)), do(36

For more information, see the [clingo Python API](https://potassco.org/clingo/python-api/5.5/).

: 